<a href="https://colab.research.google.com/github/jafetimbre/mib-fiesta/blob/master/src/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Curatarea seturilor de date



## Incarcarea seturilor de date

In [2]:
housing_columns = [ 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'condition', 'grade', 'sqft_basement', 'yr_built', 'yr_renovated' ,'zipcode', 'lat', 'long' ]
housing_data = pd.read_csv('https://raw.githubusercontent.com/jafetimbre/mib-fiesta/master/data/housing.csv', usecols=housing_columns)
housing_data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900,3,1.00,1180,5650,1.0,0,3,7,0,1955,0,98178,47.5112,-122.257
1,538000,3,2.25,2570,7242,2.0,0,3,7,400,1951,1991,98125,47.7210,-122.319
2,180000,2,1.00,770,10000,1.0,0,3,6,0,1933,0,98028,47.7379,-122.233
3,604000,4,3.00,1960,5000,1.0,0,5,7,910,1965,0,98136,47.5208,-122.393
4,510000,3,2.00,1680,8080,1.0,0,3,8,0,1987,0,98074,47.6168,-122.045


In [3]:
property_columns = [ 'BuildingTypeName', 'PropertyTypeEPA', 'City', 'Zip', 'Neighborhood', 'Latitude', 'Longitude' ]
property_data = pd.read_csv('https://raw.githubusercontent.com/jafetimbre/mib-fiesta/master/data/property_types.csv', usecols=property_columns)
property_data.head()

,BuildingTypeName,PropertyTypeEPA,City,Zip,Neighborhood,Latitude,Longitude
0,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.612190,-122.337997
1,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.613106,-122.333358
2,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.613349,-122.337699
3,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.614216,-122.336609
4,NonResidential,Hotel,SEATTLE,98121,DOWNTOWN,47.613754,-122.340924


## General

In [4]:
zip_property_data = property_data.Zip.unique()
zip_housing_data = housing_data.zipcode.unique()
zip_comon = list(set(zip_property_data).intersection(zip_housing_data))
print(zip_comon)

[98177, 98178, 98199, 98102, 98103, 98105, 98106, 98107, 98108, 98109, 98112, 98115, 98116, 98117, 98118, 98119, 98122, 98125, 98126, 98133, 98136, 98144, 98146, 98155]


## Procesarea primului set

In [5]:
property_data.head()

,BuildingTypeName,PropertyTypeEPA,City,Zip,Neighborhood,Latitude,Longitude
0,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.612190,-122.337997
1,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.613106,-122.333358
2,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.613349,-122.337699
3,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.614216,-122.336609
4,NonResidential,Hotel,SEATTLE,98121,DOWNTOWN,47.613754,-122.340924


In [6]:
property_data = property_data[property_data.BuildingTypeName.map(lambda e: e in ['NonResidential', 'Nonresidential COS', 'SPS-District K-12', 'Campus'])]
property_data = property_data[property_data.City.map(lambda e: e == 'SEATTLE')]
property_data = property_data[property_data.Zip.map(lambda e: e in zip_comon)]
property_data = property_data[property_data.PropertyTypeEPA.map(lambda e: e != 'Hotel')]

property_data = property_data.drop(['BuildingTypeName', 'City'], axis=1)
property_data = property_data.rename(columns={'PropertyTypeEPA': 'property_type', 'Zip': 'zip', 'Neighborhood':'neighborhood', 'Latitude': 'latitude', 'Longitude':'longitude'})

property_data.dropna(inplace=True)
property_data.reset_index(drop=True)

property_data.head()

property_data.to_csv('property_data_cleaned.csv', index = False, header=True)

## Procesarea setului 2

In [7]:
housing_data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900,3,1.00,1180,5650,1.0,0,3,7,0,1955,0,98178,47.5112,-122.257
1,538000,3,2.25,2570,7242,2.0,0,3,7,400,1951,1991,98125,47.7210,-122.319
2,180000,2,1.00,770,10000,1.0,0,3,6,0,1933,0,98028,47.7379,-122.233
3,604000,4,3.00,1960,5000,1.0,0,5,7,910,1965,0,98136,47.5208,-122.393
4,510000,3,2.00,1680,8080,1.0,0,3,8,0,1987,0,98074,47.6168,-122.045


In [8]:
housing_data.sqft_living = housing_data.sqft_living.apply(lambda e: round(e * 0.092))
housing_data.sqft_lot = housing_data.sqft_lot.apply(lambda e: round(e * 0.092))
housing_data.sqft_basement = housing_data.sqft_basement.apply(lambda e: round(e * 0.092))

housing_data = housing_data[housing_data.zipcode.map(lambda e: e in zip_comon)]

housing_data = housing_data.rename(columns={'sqft_living': 'm2_living', 'sqft_lot':'m2_lot', 'sqft_basement':'m2_basement', 'zipcode':'zip'})

housing_data.dropna(inplace=True)
housing_data.reset_index(drop=True)

housing_data.head()

housing_data.to_csv('housing_data_cleaned.csv', index = False, header=True)